# Imports

In [ ]:
import numpy as np
import pandas as pd

# Data load

## Load meta data csv into DF

## Random split train into train/val

## Create train/val/test folders

## Histo of classes in train/va/test

## Check missing classes from val or test

## Create ALL class folders

# Data download

In [ ]:
def download(df, data_dir):
    Loop on img_files in df
        Open class_lbl data_dir
        if file exists:
            continue
        else:
            try:
                download
                Mark file exists in df

            except:
                Mark file missing in df
            

## Train

In [ ]:
## Val

In [ ]:
## Test

In [ ]:
# Model

In [ ]:
## Top model pre-train

In [ ]:
## Top model + top base layers fine tune

In [ ]:
# Predict on test

# Prepare submission